# NMME from compiled files

Filepaths to hindcasts: https://github.com/iri-pycpt/pycpt/blob/main/pycpt/src/cptdl/catalog.py#L97

Filepaths to forecasts: https://github.com/iri-pycpt/pycpt/blob/main/pycpt/src/cptdl/catalog.py#L322

In [1]:
import xarray as xr
import xagg as xa
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
import warnings

import os
import re
import glob
from matplotlib import pyplot as plt
from cartopy import crs as ccrs
import cmocean

from funcs_support import get_params,area_mean,utility_print
dir_list = get_params()

In [12]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'aux_data'))

from catalog import hindcasts as urls_hind, forecasts as urls_fore

In [14]:
urls_hind['CFSv2.PRCP']

"https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.PENTAD_SAMPLES_FULL/.prec/S/%280000%201%20{threeletters[fdate.month]}%20{first_year}-{final_year}%29/VALUES/L/{lead_low}/{lead_high}/RANGEEDGES/%5BL%5D//keepgrids/average/Y/{predictor_extent['south']}/{predictor_extent['north']}/RANGEEDGES/X/{predictor_extent['west']}/{predictor_extent['east']}/RANGEEDGES/{'%5BM%5D/average/' if ensemblemean else ''}{seasonal_target_length(target)}/mul/-999/setmissing_value/{'%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv' if filetype == 'cptv10.tsv' and ensemblemean else '%5BX/Y%5D%5BL/S/add/M%5D/cptv10.tsv' if filetype =='cptv10.tsv' and not ensemblemean else filetype}"

In [17]:
predictor_extent = {'south':'40S','north':'40N'}
os.system(f'wget https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.PENTAD_SAMPLES_FULL/.prec/S/RANGEEDGES/%5BL%5D/keepgrids/average/Y/'+
 f'{predictor_extent['south']}/{predictor_extent['north']} '+
         f'-P {dir_list['raw']+'CFSv2/'}')

--2026-02-02 10:44:48--  https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.PENTAD_SAMPLES_FULL/.prec/S/RANGEEDGES/%5BL%5D/keepgrids/average/Y/40S/40N
Resolving iridl.ldeo.columbia.edu (iridl.ldeo.columbia.edu)... 129.236.110.88
Connecting to iridl.ldeo.columbia.edu (iridl.ldeo.columbia.edu)|129.236.110.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://iridl.ldeo.columbia.edu/auth/login?redirect=%2FSOURCES%2F.Models%2F.NMME%2F.NCEP-CFSv2%2F.HINDCAST%2F.PENTAD_SAMPLES_FULL%2F.prec%2FS%2FRANGEEDGES%2F%255BL%255D%2Fkeepgrids%2Faverage%2FY%2F40S%2F40N [following]
--2026-02-02 10:44:48--  https://iridl.ldeo.columbia.edu/auth/login?redirect=%2FSOURCES%2F.Models%2F.NMME%2F.NCEP-CFSv2%2F.HINDCAST%2F.PENTAD_SAMPLES_FULL%2F.prec%2FS%2FRANGEEDGES%2F%255BL%255D%2Fkeepgrids%2Faverage%2FY%2F40S%2F40N
Reusing existing connection to iridl.ldeo.columbia.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 4454 (4.3K) [text/htm

0

In [22]:
ds = xr.open_dataset('https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.PENTAD_SAMPLES_FULL/.prec/dods',
               decode_times=False)
ds

<xarray.Dataset> Size: 39GB
Dimensions:  (S: 530, L: 10, M: 28, Y: 181, X: 360)
Coordinates:
  * S        (S) float32 2kB 264.0 265.0 266.0 267.0 ... 790.0 791.0 792.0 793.0
  * L        (L) float32 40B 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * M        (M) float32 112B 1.0 2.0 3.0 4.0 5.0 ... 24.0 25.0 26.0 27.0 28.0
  * Y        (Y) float32 724B -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * X        (X) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
Data variables:
    prec     (S, L, M, Y, X) float32 39GB ...
Attributes:
    Conventions:  IRIDL

<xarray.DataArray 'M' (M: 28)> Size: 112B
array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.],
      dtype=float32)
Coordinates:
  * M        (M) float32 112B 1.0 2.0 3.0 4.0 5.0 ... 24.0 25.0 26.0 27.0 28.0
Attributes:
    pointwidth:  1.0
    gridtype:    0
    units:       unitless

In [27]:
ds.L

<xarray.DataArray 'L' (L: 10)> Size: 40B
array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5], dtype=float32)
Coordinates:
  * L        (L) float32 40B 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
Attributes:
    pointwidth:  1.0
    gridtype:    0
    units:       months